In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [2]:
# df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

In [3]:
df.head()

,title,date,content
0,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],2020-09-04,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
1,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,얼마에 사셨어요? 저도 관심이 있는데요.
3,[제목 : [오아시스]\n 오아시스 주식이요 ],2021-04-17,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
4,[제목 : [오아시스] 70만원이면 비싼건가요 ],2021-04-19,"30주 70만원에 살려고하는데, 비싼건가요"


## null 값 확인

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  133 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


In [5]:
# NULL 값 있는 행 확인
df[df['content'].isnull()]

,title,date,content
31,[제목 : [오아시스] 상장노력은 하는거죠? ],2021-07-09,NaN
40,[제목 : [오아시스] 조용히 갑시다~~ ],2021-07-16,NaN
43,[제목 : [오아시스] 지금 잘가고 있긴 한데 스피드 좀 올려~~ ],2021-07-29,NaN


In [6]:
# 공백으로 NULL 값 채우기
df = df.fillna('')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  136 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


내용에 빈 값이 있지만 title에서 추출할 것이 있지 않을까 해서 공백으로 채워줌

## index 날짜로 바꿔주기

In [8]:
# date를 오름차순 정렬
df.date = df.date.sort_index(ascending=True)
# index로 설정
df = df.set_index('date')
df

,title,content
date,,
2020-09-04,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],얼마에 사셨어요? 저도 관심이 있는데요.
2021-04-17,[제목 : [오아시스]\n 오아시스 주식이요 ],오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
2021-04-19,[제목 : [오아시스] 70만원이면 비싼건가요 ],"30주 70만원에 살려고하는데, 비싼건가요"
...,...,...
2022-08-31,[제목 : [오아시스] 우크라이나와 러시아 ],덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...
2022-08-31,"[제목 : [오아시스] 오늘 날짜(2022.08.31) 더벨 기사, 오아시스 기업...",코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,[제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ],연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...


## 특수문자 제거 정규표현식

In [9]:
df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_1022/1046961940.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_1022/1046961940.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [10]:
df

,title,content
date,,
2020-09-04,제목 오아시스\n 팬데믹시대에 내마음의 표준을 바꾸는 유통,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다매장에서 인테넷의...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요어제 아시는 분...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,얼마에 사셨어요 저도 관심이 있는데요
2021-04-17,제목 오아시스\n 오아시스 주식이요,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면 팔육일팔 삼육사일로연락주세요
2021-04-19,제목 오아시스 70만원이면 비싼건가요,30주 70만원에 살려고하는데 비싼건가요
...,...,...
2022-08-31,제목 오아시스 우크라이나와 러시아,덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는 길고 짧은건 대봐야 안다고우크...
2022-08-31,제목 오아시스 오늘 날짜20220831 더벨 기사 오아시스 기업가치 최대 2조 제시,코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,제목 오아시스 이 시기에 오히려 주목해야 할 기업,연일 금리가 급등하고 있다 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유니...


## 토큰화

In [11]:
okt = Okt()


### 추출했던 word_list에서 불용어들만 정리

In [ ]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [ ]:
with open('stopword_add_title.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
data

In [ ]:
stopword_add = sum(data, [])
stopword_add

In [ ]:
# 불용어 처리 

stopwords = stopword_add + ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n','\n ', '제목']
stopwords

In [ ]:
title = []
for sentence in tqdm(df['title']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_title = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    title.append(stopwords_removed_title)

In [ ]:
print(len(title))

In [ ]:
from collections import Counter

In [ ]:
# 한 날짜? 한 행당 횟수 추출

for i in range(136):
    vocab = Counter(title[i])
    print(vocab)


In [ ]:
# 전체 중 빈도수 출력

all_word_list_title = sum(title, [])
vocab = Counter(all_word_list_title)
print(vocab)

In [ ]:
word_list = list(vocab.keys())
word_list

In [ ]:
word_rank = list(vocab.values())
word_rank

In [ ]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

In [ ]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [ ]:
for i in range(26):
    print(word_data[ i*10 : (i+1)*10 ])

word_data.to_csv('sentimental(38).csv')

In [ ]:
word_data

sentimental = pd.read_csv('sentimental(38).csv').drop(columns=['Unnamed: 0'])
sentimental = sentimental.fillna('-')

sentimental

sentimental.to_csv('sentimental(38).csv')

In [ ]:
community_title_label = pd.read_csv('sentimental(38)_label_title.csv').drop(columns=['Unnamed: 0'])
community_title_label

### labeling 한 거에서 추가 불용어 처리

In [ ]:
community_title_label = community_title_label[~community_title_label['label'].str.contains('--')]
community_title_label

In [ ]:
community_title_label.to_csv('community_title_label.csv')

In [ ]:
data['label'].value_counts().plot(kind = 'bar')